<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,4015,30,0.75,185780124695
1,054620,APS홀딩스,KOSDAQ,금융,16250,550,3.50,331406091250
2,265520,AP시스템,KOSDAQ,반도체,25300,-750,-2.88,366349743100
3,211270,AP위성,KOSDAQ,통신장비,17050,-400,-2.29,257153283200
4,013720,CBI,KOSDAQ,운송장비·부품,2980,-350,-10.51,151079369500
...,...,...,...,...,...,...,...,...
1511,024060,흥구석유,KOSDAQ,유통,8110,60,0.75,121650000000
1512,010240,흥국,KOSDAQ,기계·장비,9190,-140,-1.50,113245576240
1513,189980,흥국에프엔비,KOSDAQ,음식료·담배,3250,-90,-2.69,124758162750
1514,037440,희림,KOSDAQ,기타서비스,6790,-70,-1.02,94533605250


In [7]:
'''
%%timeit
df.loc[df['시가총액'].between(30000000000,70000000000), '종목명']

%%timeit
df[df['시가총액'].between(30000000000,70000000000)]['종목명']
'''

"\n%%timeit\ndf.loc[df['시가총액'].between(30000000000,70000000000), '종목명']\n\n%%timeit\ndf[df['시가총액'].between(30000000000,70000000000)]['종목명']\n"

In [8]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1480,-5,-0.34,56142488720
20,050120,ES큐브,KOSDAQ,유통,4600,0,0.00,62394795600
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
35,099520,ITX-AI,KOSDAQ,통신장비,1285,0,0.00,37652662655
...,...,...,...,...,...,...,...,...
1464,079170,한창산업,KOSDAQ,금속,9410,-180,-1.88,48932000000
1467,002680,한탑,KOSDAQ,음식료·담배,1350,-15,-1.10,38164483800
1476,900270,헝셩그룹,KOSDAQ,금융,490,-10,-2.00,39200000000
1489,011080,형지I&C,KOSDAQ,섬유·의류,1410,0,0.00,54927491880


In [9]:
!pip install OpenDartReader

In [10]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key) 

In [11]:
'''search_start = '2021-08-20'
search_end = '2021-09-28
'''

search_start = '2021-08-01'
search_end = '2021-10-01'

In [12]:
dart_df = dart.list(start=search_start, end=search_end, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [13]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

In [14]:
def get_incomestatement(df):
  is_df = df.loc[df['sj_nm'] == '손익계산서']
  try:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03', 'current12', 'previous12']
  except:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_is_score(is_df):
  is_score = {}
  for n in is_df['name']:
    current = str_int(is_df.loc[is_df['name'] == n, 'current03'].values[0])
    previous = str_int(is_df.loc[is_df['name'] == n, 'previous03'].values[0])
    is_score[n] = get_growth_rate(previous, current)
  return is_score

In [15]:
def get_balancesheet(df):
  bs_df = df.loc[df['sj_nm'] == '재무상태표']
  t = bs_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_bs_score(bs_df):
  bs_score = {}
  for n in bs_df['name']:
    current = str_int(bs_df.loc[bs_df['name'] == n, 'current03'].values[0])
    previous = str_int(bs_df.loc[bs_df['name'] == n, 'previous03'].values[0])
    bs_score[n] = get_growth_rate(previous, current)
  return bs_score

In [16]:
def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

def str_int(s):
  return int(s.replace(',',''))

In [17]:
df = get_recent_finstate('290270')
is_df = get_incomestatement(df)
is_score = get_is_score(is_df)

bs_df = get_balancesheet(df)
bs_score = get_bs_score(bs_df)

In [36]:
for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = get_incomestatement(df)
    is_score = get_is_score(is_df)

    bs_df = get_balancesheet(df)
    bs_score = get_bs_score(bs_df)

    date = out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0]
    name = out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0]
    print(date)
    print(name)
    print(is_score)
    print(bs_score)
    print()

    t = is_score
    t.update(bs_score)
    score_dic[name] = t
  except:
    print('ERROR', name)
    print()

20210917
이큐셀
{'매출액': 0.5946777575625294, '영업이익': 0.7633451325396896, '법인세차감전 순이익': 0.9813018644484682, '당기순이익': 0.9721316919875065}
{'유동자산': -0.2370703695783634, '비유동자산': -0.24581652782162192, '자산총계': -0.24148035427064782, '유동부채': -0.7701239556796861, '비유동부채': 1.41164381896257, '부채총계': -0.502050240114592, '자본금': -0.5427179216068487, '이익잉여금': -0.006226826540301651, '자본총계': 3.0341337394145134}

20210917
스카이이앤엠
{'매출액': -0.14140310196929798, '영업이익': 0.19975064286302283, '법인세차감전 순이익': -0.02171366431663808, '당기순이익': -0.025982011078868078}
{'유동자산': 0.281189948339449, '비유동자산': -0.27040936858142745, '자산총계': -0.10365077472303555, '유동부채': -0.43302546694734423, '비유동부채': -0.07561410483365145, '부채총계': -0.42283730207782316, '자본금': 0.4928203766130843, '이익잉여금': -0.33046473675636434, '자본총계': 0.7134420473245608}

20210917
포시에스
{'매출액': 0.09241148397369084, '영업이익': -0.011265960247431392, '법인세차감전 순이익': -0.20417519335780188, '당기순이익': -0.16989095071462781}
{'유동자산': 0.15750562429626755, '비유동자산': 0.079073301409

In [43]:
score_df = pd.DataFrame(score_dic).T
score_df

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
이큐셀,0.594678,0.763345,0.981302,0.972132,-0.237070,-0.245817,-0.241480,-0.770124,1.411644,-0.502050,-0.542718,-0.006227,3.034134
스카이이앤엠,-0.141403,0.199751,-0.021714,-0.025982,0.281190,-0.270409,-0.103651,-0.433025,-0.075614,-0.422837,0.492820,-0.330465,0.713442
포시에스,0.092411,-0.011266,-0.204175,-0.169891,0.157506,0.079073,0.108244,0.080759,0.074166,0.080049,0.048258,0.097788,0.109553
아세아텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
그리티,-0.167455,-0.538194,-0.590885,-0.456222,0.011179,0.609790,0.106131,0.543526,0.441495,0.520342,0.000000,-0.050226,0.017878
한솔인티큐브,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
부스타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
토박스코리아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
score_df.sort_values(by=['매출액'], ascending=False)

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
KD,32.208103,0.496957,1.461314,1.277034,0.208093,0.076718,0.121690,0.126780,0.126522,0.126615,0.000000,0.076598,0.106348
마이더스AI,11.877421,0.674781,-0.299459,-0.301819,0.315613,0.792098,0.495103,3.376664,-0.169787,3.047695,0.176382,-0.301769,0.030572
엔시트론,2.708086,1.365504,0.764400,0.764400,1.990550,-0.620493,0.228568,2.406202,-0.967482,0.418582,0.000000,-0.011532,0.104463
아스타,2.535109,-0.246639,-0.391328,-6.460217,-0.270148,-0.091022,-0.207385,-0.267620,0.379494,0.095370,0.000000,-0.055875,-0.295498
골드앤에스,2.415616,1.415873,1.295806,1.309928,0.283288,0.043428,0.148574,0.246654,0.545679,0.440468,0.000000,0.006438,0.022152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
아이컴포넌트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
한솔인티큐브,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
부스타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
토박스코리아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
score_df.sort_values(by=['영업이익'], ascending=False)

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
신진에스엠,0.384187,20.137929,13.237856,88.879681,0.094614,-0.078565,0.009690,0.120264,-0.134514,0.009498,0.000000,0.014656,0.009811
빛샘전자,1.245450,17.850939,10.369221,33.479630,0.027221,-0.001433,0.016157,0.066732,-0.359900,-0.051883,0.038476,0.020871,0.034880
티피씨글로벌,0.202120,12.543472,-0.262200,-0.437397,0.162128,-0.011566,0.061633,0.077606,-0.544045,-0.128867,0.034248,0.027364,0.207536
에이치케이,0.314519,12.525597,51.430865,5.089385,0.048257,0.013365,0.035233,-0.008065,0.332062,-0.001622,0.000000,0.093286,0.059681
...,...,...,...,...,...,...,...,...,...,...,...,...,...
아이컴포넌트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
한솔인티큐브,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
부스타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
토박스코리아,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
